<a href="https://colab.research.google.com/github/frankplus/transformer-chatbot/blob/t2t_evolved_transformer/t2t_chatbot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Initialization

In [1]:
!pip install -q -U tensorflow-gpu==1.15.2
!pip install -q -U tensorflow-datasets==3.2.1
!pip install -q -U tensor2tensor

import tensorflow as tf
import os

from google.colab import drive
drive.mount('/gdrive')

project_dir = "/gdrive/MyDrive/transformer-chatbot/"
MODEL_DIR = project_dir + "saved_model/t2t_chatbot/"
DATASET_DIR = project_dir + "conversational-dataset/"

!mkdir -p $DATASET_DIR
!mkdir -p $MODEL_DIR

Drive already mounted at /gdrive; to attempt to forcibly remount, call drive.mount("/gdrive", force_remount=True).


## Hyperparameters

In [2]:
MAX_SAMPLES = 1000000
DATA_DIR = MODEL_DIR + 'data'
TMP_DIR = MODEL_DIR + 'tmp'
TRAIN_DIR = MODEL_DIR + 'train'
PROBLEM = 'chat_bot'

USE_TPU = False
MODEL = "evolved_transformer"
HPARAMS = "evolved_transformer_base"
NUM_ENCODER_LAYERS = 1
NUM_DECODER_LAYERS = 4
BATCH_SIZE = 2048
MAX_LENGTH = 40
OPTIMIZER = 'adam'
VOCAB_SIZE = 2**13

CONVERSATION_TURNS = 3

TRAIN_STEPS = 100000 # Total number of train steps for all Epochs
EVAL_STEPS = 100 # Number of steps to perform for each evaluation
SAVE_CHECKPOINTS_STEPS = 5000
KEEP_CHECKPOINT_MAX = 1

## Problem definition

In [3]:
from tensor2tensor.data_generators import problem
from tensor2tensor.data_generators import text_problems
from tensor2tensor.utils import registry
from tensor2tensor.data_generators import text_encoder
from collections import deque
import re

PATH_TO_DATASET = DATASET_DIR + 'it'

if not os.path.isfile(PATH_TO_DATASET):
    path_to_zip = tf.keras.utils.get_file(
        DATASET_DIR + "it.gz",
        origin='http://opus.nlpl.eu/download.php?f=OpenSubtitles/v2018/mono/OpenSubtitles.it.gz')

    !gzip -dk $path_to_zip

def preprocess_sentence(sentence):
    sentence = sentence.lower().strip()
    sentence = re.sub(r"[^a-zA-Z0-9?.!,àèìòùáéíóú']+", " ", sentence)
    sentence = sentence.replace(" ' ", " ")
    sentence = sentence.strip()
    return sentence

@registry.register_problem
class ChatBot(text_problems.Text2TextProblem):
    @property
    def approx_vocab_size(self):
        return VOCAB_SIZE
    
    @property
    def is_generate_per_split(self):
        return False
 
    @property
    def dataset_splits(self):
        return [{
            "split": problem.DatasetSplit.TRAIN,
            "shards": 9,
        }, {
            "split": problem.DatasetSplit.EVAL,
            "shards": 1,
        }]

    SENTENCE_SEPARATOR = "<SEP>"
    SENTENCE_SEPARATOR_ID = 2

    @property
    def additional_reserved_tokens(self):
        return [self.SENTENCE_SEPARATOR]
 
    def generate_samples(self, data_dir, tmp_dir, dataset_split):
        conversation = deque()
        count = 0
        with open(PATH_TO_DATASET, 'r') as file:
            conversation.append(preprocess_sentence(file.readline()))
            line = file.readline()
            while line and count < MAX_SAMPLES:
                preprocessed = preprocess_sentence(line)
                conversation.append(preprocessed)
                if len(conversation) > CONVERSATION_TURNS + 1:
                    conversation.popleft()
                yield {'inputs': " $ ".join(list(conversation)[:-1]), 'targets': conversation[-1]}
                line = file.readline()
                count += 1

    def generate_encoded_samples(self, data_dir, tmp_dir, dataset_split):

        generator = self.generate_samples(data_dir, tmp_dir, dataset_split)
        encoder = self.get_or_create_vocab(data_dir, tmp_dir)

        def generate_encoded(generator, encoder):
            count = 0
            for sample in generator:
                encoded_inputs = []
                for conversation_turn in sample["inputs"].split(" $ "):
                    encoded_inputs.extend(encoder.encode(conversation_turn))
                    encoded_inputs.append(self.SENTENCE_SEPARATOR_ID)
                encoded_inputs.pop()
                encoded_inputs.append(text_encoder.EOS_ID)
                if len(encoded_inputs) > MAX_LENGTH:
                    encoded_inputs = encoded_inputs[-MAX_LENGTH:]
                sample["inputs"] = encoded_inputs
                sample["targets"] = encoder.encode(sample["targets"])
                sample["targets"].append(text_encoder.EOS_ID)
                if count > 100 and count < 110:
                    print("_______INPUT_______")
                    print(encoder.decode(sample["inputs"]))
                    print("_______TARGET_______")
                    print(encoder.decode(sample["targets"]))
                count += 1
                yield sample
        return generate_encoded(generator, encoder)

## Generate data

In [4]:
from tensor2tensor import problems

t2t_problem = problems.problem(PROBLEM)
t2t_problem.generate_data(DATA_DIR, TMP_DIR)

INFO:tensorflow:Generating vocab file: /gdrive/MyDrive/transformer-chatbot/saved_model/t2t_chatbot/data/vocab.chat_bot.8192.subwords


INFO:tensorflow:Generating vocab file: /gdrive/MyDrive/transformer-chatbot/saved_model/t2t_chatbot/data/vocab.chat_bot.8192.subwords


INFO:tensorflow:Trying min_count 500


INFO:tensorflow:Trying min_count 500


INFO:tensorflow:Iteration 0


...

INFO:tensorflow:read: 100000


INFO:tensorflow:read: 100000


INFO:tensorflow:read: 100000


INFO:tensorflow:Data shuffled.


INFO:tensorflow:Data shuffled.


## Training

In [ ]:
from tensor2tensor.utils.trainer_lib import create_run_config, create_experiment
from tensor2tensor.utils.trainer_lib import create_hparams
from tensor2tensor.utils import registry, hparam
from tensor2tensor import models
from tensor2tensor import problems
import json

# Init Hparams object from T2T Problem
hparams = create_hparams(HPARAMS)

# Make Changes to Hparams
hparams.num_encoder_layers = NUM_ENCODER_LAYERS
hparams.num_decoder_layers = NUM_DECODER_LAYERS
hparams.batch_size = BATCH_SIZE
hparams.max_length = MAX_LENGTH
hparams.optimizer = OPTIMIZER

hparams_json = hparams.to_json()
print(str(hparams_json))

# Save hparams 
with open(MODEL_DIR + 'hparams.json', 'w') as json_file:
    json_file.write(hparams_json)

RUN_CONFIG = create_run_config(
      model_dir=TRAIN_DIR,
      model_name=MODEL,
      save_checkpoints_steps = SAVE_CHECKPOINTS_STEPS,
      keep_checkpoint_max = KEEP_CHECKPOINT_MAX
)

tensorflow_exp_fn = create_experiment(
        run_config=RUN_CONFIG,
        hparams=hparams,
        model_name=MODEL,
        problem_name=PROBLEM,
        data_dir=DATA_DIR, 
        train_steps=TRAIN_STEPS, 
        eval_steps=EVAL_STEPS, 
        use_tpu=USE_TPU,
        schedule="continuous_train_and_eval",
        eval_throttle_seconds=300,
        use_xla=True # For acceleration
    ) 

tensorflow_exp_fn.continuous_train_and_eval()

{"batch_size": 2048, "batch_shuffle_size": 512, "use_fixed_batch_size": false, "num_hidden_layers": 6, "kernel_height": 3, "kernel_width": 1, "hidden_size": 512, "compress_steps": 0, "dropout": 0.2, "clip_grad_norm": 0.0, "grad_noise_scale": 0.0, "summarize_grads": false, "mlperf_mode": false, "summarize_vars": false, "initializer": "uniform_unit_scaling", "initializer_gain": 1.0, "label_smoothing": 0.1, "optimizer": "adam", "optimizer_adam_epsilon": 1e-09, "optimizer_adam_beta1": 0.9, "optimizer_adam_beta2": 0.997, "optimizer_momentum_momentum": 0.9, "optimizer_momentum_nesterov": false, "optimizer_adafactor_beta1": 0.0, "optimizer_adafactor_beta2": 0.999, "optimizer_adafactor_factored": true, "optimizer_adafactor_decay_type": "pow", "optimizer_adafactor_memory_exponent": 0.8, "optimizer_adafactor_clipping_threshold": 1.0, "optimizer_adafactor_multiply_by_parameter_scale": true, "optimizer_multistep_accumulate_steps": 0, "mixed_precision_optimizer_loss_scaler": "exponential", "mixed_p

Instructions for updating:
When switching to tf.estimator.Estimator, use tf.estimator.RunConfig instead.


INFO:tensorflow:Configuring DataParallelism to replicate the model.


INFO:tensorflow:Configuring DataParallelism to replicate the model.


INFO:tensorflow:schedule=continuous_train_and_eval


...

INFO:tensorflow:loss = 2.3873317, step = 96800 (24.407 sec)


INFO:tensorflow:global_step/sec: 4.00306


INFO:tensorflow:global_step/sec: 4.00306


INFO:tensorflow:loss = 2.399053, step = 96900 (24.975 sec)


INFO:tensorflow:loss = 2.399053, step = 96900 (24.975 sec)


## Predict

In [ ]:
import tensorflow as tf
from tensor2tensor import models
from tensor2tensor import problems
from tensor2tensor.utils import hparams_lib
from tensor2tensor.utils import registry
from tensor2tensor.data_generators import text_problems
import numpy as np
import re

tfe = tf.contrib.eager
tfe.enable_eager_execution()
Modes = tf.estimator.ModeKeys

chat_bot_problem = problems.problem("chat_bot")
encoders = chat_bot_problem.feature_encoders(DATA_DIR)
hparams = hparams_lib.create_hparams_from_json(MODEL_DIR + 'hparams.json')
hparams.data_dir = DATA_DIR
hparams_lib.add_problem_hparams(hparams, "chat_bot")
ckpt_path = tf.train.latest_checkpoint(TRAIN_DIR)

chatbot_model = registry.model(MODEL)(hparams, Modes.PREDICT)

def encode(conversation, output_str=None):
    """Input str to features dict, ready for inference"""
    encoded_inputs = []
    for conversation_turn in conversation:
        encoded_inputs += encoders["inputs"].encode(conversation_turn) + [2]
    encoded_inputs.pop()
    encoded_inputs += [1]
    if len(encoded_inputs) > hparams.max_length:
        encoded_inputs = encoded_inputs[-hparams.max_length:]
    batch_inputs = tf.reshape(encoded_inputs, [1, -1, 1])  # Make it 3D.
    return {"inputs": batch_inputs}

def decode(integers):
    """List of ints to str"""
    if 1 in integers:
        integers = integers[:integers.index(1)]
    return encoders["inputs"].decode(integers)

def preprocess_sentence(sentence):
  sentence = sentence.lower().strip()
  # creating a space between a word and the punctuation following it
  # eg: "he is a boy." => "he is a boy ."
  sentence = re.sub(r"([?.!,])", r" \1 ", sentence)
  sentence = sentence.replace("'", "' ")
  sentence = re.sub(r'[" "]+', " ", sentence)
  sentence = re.sub(r"[^a-zA-Z0-9?.!,àèìòùáéíóú']+", " ", sentence)
  sentence = sentence.strip()
  return sentence

def predict(conversation):
    preprocessed = [preprocess_sentence(x) for x in conversation]
    encoded_inputs = encode(preprocessed)
    with tfe.restore_variables_on_create(ckpt_path):
        model_output = chatbot_model.infer(encoded_inputs, beam_size=20, top_beams=20)["outputs"]
    responses = [decode(list(response)) for response in np.squeeze(model_output)]

    # pick a response with higher probability to longer responses 
    tot_lengths = sum([len(x) for x in responses])
    return np.random.choice(responses, p = [len(x)/tot_lengths for x in responses])


conversation = []
while True:
    sentence = input("Input: ")
    conversation.append(sentence)
    while len(conversation) > CONVERSATION_TURNS: 
        conversation.pop(0)
    response = predict(conversation)
    conversation.append(response)
    print(response)

## Tensorboard

In [ ]:
%load_ext tensorboard
%tensorboard --logdir $TRAIN_DIR